In [1]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
#driver = webdriver.Chrome(executable_path="C:\Drivers\chromedriver")
#driver.get("https://nregarep2.nic.in/netnrega/dynamic2/DynamicReport_new4.aspx")
from selenium.webdriver.common.action_chains import ActionChains
#driver.maximize_window()
#driver.implicitly_wait(10)
#wait = WebDriverWait(driver, 10)
import select as select
#time.sleep(5) 

def get_manrega_df( driver, download_dir, countrycode, hslevelcode, currency):
    url = 'https://nregarep2.nic.in/netnrega/dynamic2/DynamicReport_new4.aspx'      #Url need to scrape data from
    driver.get(url)                                                 #Initializing driver with the url
    driver.implicitly_wait(10)

    check_list = ['ChkLstFieldsWorkerA_0', 'ChkLstFieldsWorkerA_1','ChkLstFieldsWorkerA_2','ChkLstFieldsWorkerA_3','ChkLstFieldsWorkerA_4',
                 'ChkLstFieldsWorkerC_0', 'ChkLstFieldsWorkerC_1', 'ChkLstFieldsWorkerD_0', 'ChkLstFieldsWorkerD_1',
                  'ChkLstFieldsWorkerE_0','ChkLstFieldsWorkerE_1','ChkLstFieldsWorkerE_2','ChkLstFieldsWorkerE_3',
                  'ChkLstFieldsWorkerE_4','ChkLstFieldsWorkerE_5','ChkLstFieldsWorkerE_6', 
                 'ChkLstFieldsWorkerE_7','ChkLstFieldsWorkerE_8','ChkLstFieldsWorkerE_9','ChkLstFieldsWorkerE_10',
                  'ChkLstFieldsWorkerE_11','ChkLstFieldsWorkerE_12','ChkLstFieldsWorkerE_13',
                  'CTxtBox1', 'CTxtBox2', 
                 'ChkLstFieldsWork_0', 'ChkLstFieldsWork_1','ChkLstFieldsWork_2','ChkLstFieldsWork_3',
                 'ChkLstFieldsWork_W_0','ChkLstFieldsWork_W_1',
                 'ChkBox1','ChkBox2','ChkBox4',
                 'CTxtBox3','CTxtBox4','CTxtBox5','CTxtBox6','CTxtBox7','CTxtBox8','CTxtBox9','CTxtBox10']

    year_slct = ['2011-2012', '2012-2013','2013-2014','2014-2015','2015-2016','2016-2017','2017-2018','2018-2019','2019-2020','2020-2021']

    lod_slct = ['State', 'District', 'Block']

    total_hh = ['1-10', '11-20', '21-30', '31-40', '41-50', '51-60','61-70', '71-80', '81-99', '100', '>100', '>14']
    total_worker_age = ['18-30 yrs','30-40 yrs','40-50 yrs','50-60 yrs','>60 yrs']
    months = ['April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December', 'January','February', 'March']
    rem_check = ['TxtBox1', 'TxtBox9', 'TxtBox5', 'TxtBox6', 'TxtBox7', 'TxtBox8']
    drp_box = ['DdlstTxtBox1', 'DdlstTxtBox9','DdlstTxtBox5','DdlstTxtBox6','DdlstTxtBox7','DdlstTxtBox8']


    def slct_indicators():
        for i in check_list:
            checkboxes = driver.find_elements_by_id(i)
            for checkbox in checkboxes:
                while not checkbox.is_selected():
                    checkbox.click()

    def slct_state():
        lod_element = driver.find_element_by_id('regionselect')
        lod_drp = Select(lod_element)
        lod_drp.select_by_visible_text('State')
        driver.find_element_by_xpath("//*[@id='checklist']/ul/li/label/input").click()


    def slct_district():
        lod_element = driver.find_element_by_id('regionselect')
        lod_drp = Select(lod_element)
        lod_drp.select_by_visible_text('District')
        driver.find_element_by_xpath("//*[@id='checklist']/ul/li/label/input").click()


    def slct_block():
        lod_element = driver.find_element_by_id('regionselect')
        lod_drp = Select(lod_element)
        lod_drp.select_by_visible_text('Block')
        driver.find_element_by_xpath("//*[@id='checklist']/ul/li/label/input").click()

    state_table=[]
    for i in year_slct:
        slct_indicators()
        slct_state()

        year_element = driver.find_element_by_id("DdlstFinYear")
        year_drp = Select(year_element)
        year_drp.select_by_visible_text(i)

        driver.find_element_by_id("viewDummy").click()
        time.sleep(5) 

        html=driver.page_source
        soup=BeautifulSoup(html,'html.parser')
        div=soup.select_one("div#content")
        table=pd.read_html(str(div))
        state_table.append(table)
        driver.find_element_by_id("BtnBack").click()

    district_table=[]
    for i in year_slct:
        slct_indicators()
        slct_district()

        year_element = driver.find_element_by_id("DdlstFinYear")
        year_drp = Select(year_element)
        year_drp.select_by_visible_text(i)

        driver.find_element_by_id("viewDummy").click()
        time.sleep(10) 

        html=driver.page_source
        soup=BeautifulSoup(html,'html.parser')
        div=soup.select_one("div#content")
        table=pd.read_html(str(div))
        district_table.append(table)
        driver.find_element_by_id("BtnBack").click()

    block_table=[] 
    for i in check_list:
        for p in year_slct:
            driver.find_element_by_id(i).click()
            slct_block()

            year_element = driver.find_element_by_id("DdlstFinYear")
            year_drp = Select(year_element)
            year_drp.select_by_visible_text(p)

            driver.find_element_by_id("viewDummy").click()
            time.sleep(5)

            html=driver.page_source
            soup=BeautifulSoup(html,'html.parser')
            div=soup.select_one("div#content")
            table=pd.read_html(str(div))
            block_table.append(table)
            driver.find_element_by_id("BtnBack").click()

    '''df_state = pd.DataFrame(state_table)
    df_state.to_csv('df_state.csv') 

    df_district = pd.DataFrame(district_table)
    df_district.to_csv('df_district.csv')

    df_block = pd.DataFrame(block_table)
    df_block.to_csv('df_block.csv')'''

    # State wise remaining indicators
    #Total households completed person-days in range
    total_hh_completed_person_days_st=[]
    for i in year_slct:    
        for l in total_hh:

            year_element = driver.find_element_by_id("DdlstFinYear")
            year_drp = Select(year_element)
            year_drp.select_by_visible_text(i)

            slct_state()

            driver.find_element_by_id('TxtBox1').click()

            drp_ele = driver.find_element_by_id('DdlstTxtBox1')
            drp = Select(drp_ele)
            drp.select_by_visible_text(l)

            driver.find_element_by_id("viewDummy").click()
            time.sleep(2) 

            html=driver.page_source
            soup=BeautifulSoup(html,'html.parser')
            div=soup.select_one("div#content")
            table=pd.read_html(str(div))
            total_hh_completed_person_days_st.append(table)
            driver.find_element_by_id("BtnBack").click()

    #total workers employed in age group
    total_workers_employed_st=[]
    for i in year_slct:    
        for l in total_worker_age:

            year_element = driver.find_element_by_id("DdlstFinYear")
            year_drp = Select(year_element)
            year_drp.select_by_visible_text(i)

            slct_state()

            driver.find_element_by_id('TxtBox9').click()

            drp_ele = driver.find_element_by_id('DdlstTxtBox9')
            drp = Select(drp_ele)
            drp.select_by_visible_text(l)

            driver.find_element_by_id("viewDummy").click()
            time.sleep(2) 

            html=driver.page_source
            soup=BeautifulSoup(html,'html.parser')
            div=soup.select_one("div#content")
            table=pd.read_html(str(div))
            total_workers_employed_st.append(table)
            driver.find_element_by_id("BtnBack").click()

    #Total_hh_demanded_work
    total_hh_demanded_work_st=[]
    for i in year_slct:    
        for l in months:

            year_element = driver.find_element_by_id("DdlstFinYear")
            year_drp = Select(year_element)
            year_drp.select_by_visible_text(i)

            slct_state()

            driver.find_element_by_id('TxtBox5').click()

            drp_ele = driver.find_element_by_id('DdlstTxtBox5')
            drp = Select(drp_ele)
            drp.select_by_visible_text(l)

            driver.find_element_by_id("viewDummy").click()
            time.sleep(2) 

            html=driver.page_source
            soup=BeautifulSoup(html,'html.parser')
            div=soup.select_one("div#content")
            table=pd.read_html(str(div))
            total_hh_demanded_work_st.append(table)
            driver.find_element_by_id("BtnBack").click()

    #total_persons_demanded_work
    total_persons_demanded_work_st=[]
    for i in year_slct:    
        for l in months:

            year_element = driver.find_element_by_id("DdlstFinYear")
            year_drp = Select(year_element)
            year_drp.select_by_visible_text(i)

            slct_state()

            driver.find_element_by_id('TxtBox6').click()

            drp_ele = driver.find_element_by_id('DdlstTxtBox6')
            drp = Select(drp_ele)
            drp.select_by_visible_text(l)

            driver.find_element_by_id("viewDummy").click()
            time.sleep(2) 

            html=driver.page_source
            soup=BeautifulSoup(html,'html.parser')
            div=soup.select_one("div#content")
            table=pd.read_html(str(div))
            total_persons_demanded_work_st.append(table)
            driver.find_element_by_id("BtnBack").click()

    #total_hh_worked
    total_hh_worked_st=[]
    for i in year_slct:    
        for l in months:

            year_element = driver.find_element_by_id("DdlstFinYear")
            year_drp = Select(year_element)
            year_drp.select_by_visible_text(i)

            slct_state()

            driver.find_element_by_id('TxtBox7').click()

            drp_ele = driver.find_element_by_id('DdlstTxtBox7')
            drp = Select(drp_ele)
            drp.select_by_visible_text(l)

            driver.find_element_by_id("viewDummy").click()
            time.sleep(2) 

            html=driver.page_source
            soup=BeautifulSoup(html,'html.parser')
            div=soup.select_one("div#content")
            table=pd.read_html(str(div))
            total_hh_worked_st.append(table)
            driver.find_element_by_id("BtnBack").click()

    #total_persons_worked
    total_persons_worked_st=[]
    for i in year_slct:    
        for l in months:

            year_element = driver.find_element_by_id("DdlstFinYear")
            year_drp = Select(year_element)
            year_drp.select_by_visible_text(i)

            slct_state()

            driver.find_element_by_id('TxtBox8').click()

            drp_ele = driver.find_element_by_id('DdlstTxtBox8')
            drp = Select(drp_ele)
            drp.select_by_visible_text(l)

            driver.find_element_by_id("viewDummy").click()
            time.sleep(2) 

            html=driver.page_source
            soup=BeautifulSoup(html,'html.parser')
            div=soup.select_one("div#content")
            table=pd.read_html(str(div))
            total_persons_worked_st.append(table)
            driver.find_element_by_id("BtnBack").click()


    '''df_total_hh_completed_person_days_st = pd.DataFrame(total_hh_completed_person_days_st)
    df_total_hh_completed_person_days_st.to_csv('df_total_hh_completed_person_days_st.csv') 

    df_total_workers_employed_st = pd.DataFrame(total_workers_employed_st)
    df_total_workers_employed_st.to_csv('df_total_workers_employed_st.csv')

    df_total_hh_demanded_work_st = pd.DataFrame(total_hh_demanded_work_st)
    df_total_hh_demanded_work_st.to_csv('df_total_hh_demanded_work_st.csv')

    df_total_persons_demanded_work_st = pd.DataFrame(total_persons_demanded_work_st)
    df_total_persons_demanded_work_st.to_csv('df_total_persons_demanded_work_st.csv')

    df_total_hh_worked_st = pd.DataFrame(total_hh_worked_st)
    df_total_hh_worked_st.to_csv('df_total_hh_worked_st.csv')

    df_total_persons_worked_st = pd.DataFrame(total_persons_worked_st)
    df_total_persons_worked_st.to_csv('df_total_persons_worked_st.csv')'''


    # district wise remaining indicators
    #Total households completed person-days in range
    total_hh_completed_person_days_dt=[]
    for i in year_slct:    
        for l in total_hh:

            year_element = driver.find_element_by_id("DdlstFinYear")
            year_drp = Select(year_element)
            year_drp.select_by_visible_text(i)

            slct_district()

            driver.find_element_by_id('TxtBox1').click()

            drp_ele = driver.find_element_by_id('DdlstTxtBox1')
            drp = Select(drp_ele)
            drp.select_by_visible_text(l)

            driver.find_element_by_id("viewDummy").click()
            time.sleep(2) 

            html=driver.page_source
            soup=BeautifulSoup(html,'html.parser')
            div=soup.select_one("div#content")
            table=pd.read_html(str(div))
            total_hh_completed_person_days_dt.append(table)
            driver.find_element_by_id("BtnBack").click()

    #total workers employed in age group
    total_workers_employed_dt=[]
    for i in year_slct:    
        for l in total_worker_age:

            year_element = driver.find_element_by_id("DdlstFinYear")
            year_drp = Select(year_element)
            year_drp.select_by_visible_text(i)

            slct_district()

            driver.find_element_by_id('TxtBox9').click()

            drp_ele = driver.find_element_by_id('DdlstTxtBox9')
            drp = Select(drp_ele)
            drp.select_by_visible_text(l)

            driver.find_element_by_id("viewDummy").click()
            time.sleep(2) 

            html=driver.page_source
            soup=BeautifulSoup(html,'html.parser')
            div=soup.select_one("div#content")
            table=pd.read_html(str(div))
            total_workers_employed_dt.append(table)
            driver.find_element_by_id("BtnBack").click()

    #Total_hh_demanded_work
    total_hh_demanded_work_dt=[]
    for i in year_slct:    
        for l in months:

            year_element = driver.find_element_by_id("DdlstFinYear")
            year_drp = Select(year_element)
            year_drp.select_by_visible_text(i)

            slct_district()

            driver.find_element_by_id('TxtBox5').click()

            drp_ele = driver.find_element_by_id('DdlstTxtBox5')
            drp = Select(drp_ele)
            drp.select_by_visible_text(l)

            driver.find_element_by_id("viewDummy").click()
            time.sleep(2) 

            html=driver.page_source
            soup=BeautifulSoup(html,'html.parser')
            div=soup.select_one("div#content")
            table=pd.read_html(str(div))
            total_hh_demanded_work_dt.append(table)
            driver.find_element_by_id("BtnBack").click()

    #total_persons_demanded_work
    total_persons_demanded_work_dt=[]
    for i in year_slct:    
        for l in months:

            year_element = driver.find_element_by_id("DdlstFinYear")
            year_drp = Select(year_element)
            year_drp.select_by_visible_text(i)

            slct_district()

            driver.find_element_by_id('TxtBox6').click()

            drp_ele = driver.find_element_by_id('DdlstTxtBox6')
            drp = Select(drp_ele)
            drp.select_by_visible_text(l)

            driver.find_element_by_id("viewDummy").click()
            time.sleep(2) 

            html=driver.page_source
            soup=BeautifulSoup(html,'html.parser')
            div=soup.select_one("div#content")
            table=pd.read_html(str(div))
            total_persons_demanded_work_dt.append(table)
            driver.find_element_by_id("BtnBack").click()

    #total_hh_worked
    total_hh_worked_dt=[]
    for i in year_slct:    
        for l in months:

            year_element = driver.find_element_by_id("DdlstFinYear")
            year_drp = Select(year_element)
            year_drp.select_by_visible_text(i)

            slct_district()

            driver.find_element_by_id('TxtBox7').click()

            drp_ele = driver.find_element_by_id('DdlstTxtBox7')
            drp = Select(drp_ele)
            drp.select_by_visible_text(l)

            driver.find_element_by_id("viewDummy").click()
            time.sleep(2) 

            html=driver.page_source
            soup=BeautifulSoup(html,'html.parser')
            div=soup.select_one("div#content")
            table=pd.read_html(str(div))
            total_hh_worked_dt.append(table)
            driver.find_element_by_id("BtnBack").click()

    #total_persons_worked
    total_persons_worked_dt=[]
    for i in year_slct:    
        for l in months:

            year_element = driver.find_element_by_id("DdlstFinYear")
            year_drp = Select(year_element)
            year_drp.select_by_visible_text(i)

            slct_district()

            driver.find_element_by_id('TxtBox8').click()

            drp_ele = driver.find_element_by_id('DdlstTxtBox8')
            drp = Select(drp_ele)
            drp.select_by_visible_text(l)

            driver.find_element_by_id("viewDummy").click()
            time.sleep(2) 

            html=driver.page_source
            soup=BeautifulSoup(html,'html.parser')
            div=soup.select_one("div#content")
            table=pd.read_html(str(div))
            total_persons_worked_tt.append(table)
            driver.find_element_by_id("BtnBack").click()

    '''df_total_hh_completed_person_days_dt = pd.DataFrame(total_hh_completed_person_days_dt)
    df_total_hh_completed_person_days_dt.to_csv('df_total_hh_completed_person_days_dt.csv') 

    df_total_workers_employed_dt = pd.DataFrame(total_workers_employed_dt)
    df_total_workers_employed_dt.to_csv('df_total_workers_employed_dt.csv')

    df_total_hh_demanded_work_dt = pd.DataFrame(total_hh_demanded_work_dt)
    df_total_hh_demanded_work_dt.to_csv('df_total_hh_demanded_work_dt.csv')

    df_total_persons_demanded_work_dt = pd.DataFrame(total_persons_demanded_work_dt)
    df_total_persons_demanded_work_dt.to_csv('df_total_persons_demanded_work_dt.csv')

    df_total_hh_worked_dt = pd.DataFrame(total_hh_worked_dt)
    df_total_hh_worked_dt.to_csv('df_total_hh_worked_dt.csv')

    df_total_persons_worked_dt = pd.DataFrame(total_persons_worked_dt)
    df_total_persons_worked_dt.to_csv('df_total_persons_worked_dt.csv')
    '''
    # Block wise remaining indicators
    #Total households completed person-days in range
    total_hh_completed_person_days_bl=[]
    for i in year_slct:    
        for l in total_hh:

            year_element = driver.find_element_by_id("DdlstFinYear")
            year_drp = Select(year_element)
            year_drp.select_by_visible_text(i)

            slct_block()

            driver.find_element_by_id('TxtBox1').click()

            drp_ele = driver.find_element_by_id('DdlstTxtBox1')
            drp = Select(drp_ele)
            drp.select_by_visible_text(l)

            driver.find_element_by_id("viewDummy").click()
            time.sleep(2) 

            html=driver.page_source
            soup=BeautifulSoup(html,'html.parser')
            div=soup.select_one("div#content")
            table=pd.read_html(str(div))
            total_hh_completed_person_days_bl.append(table)
            driver.find_element_by_id("BtnBack").click()

    #total workers employed in age group
    total_workers_employed_bl=[]
    for i in year_slct:    
        for l in total_worker_age:

            year_element = driver.find_element_by_id("DdlstFinYear")
            year_drp = Select(year_element)
            year_drp.select_by_visible_text(i)

            slct_block()

            driver.find_element_by_id('TxtBox9').click()

            drp_ele = driver.find_element_by_id('DdlstTxtBox9')
            drp = Select(drp_ele)
            drp.select_by_visible_text(l)

            driver.find_element_by_id("viewDummy").click()
            time.sleep(2) 

            html=driver.page_source
            soup=BeautifulSoup(html,'html.parser')
            div=soup.select_one("div#content")
            table=pd.read_html(str(div))
            total_workers_employed_bl.append(table)
            driver.find_element_by_id("BtnBack").click()

    #Total_hh_demanded_work
    total_hh_demanded_work_bl=[]
    for i in year_slct:    
        for l in months:

            year_element = driver.find_element_by_id("DdlstFinYear")
            year_drp = Select(year_element)
            year_drp.select_by_visible_text(i)

            slct_block()

            driver.find_element_by_id('TxtBox5').click()

            drp_ele = driver.find_element_by_id('DdlstTxtBox5')
            drp = Select(drp_ele)
            drp.select_by_visible_text(l)

            driver.find_element_by_id("viewDummy").click()
            time.sleep(2) 

            html=driver.page_source
            soup=BeautifulSoup(html,'html.parser')
            div=soup.select_one("div#content")
            table=pd.read_html(str(div))
            total_hh_demanded_work_bl.append(table)
            driver.find_element_by_id("BtnBack").click()

    #total_persons_demanded_work
    total_persons_demanded_work_bl=[]
    for i in year_slct:    
        for l in months:

            year_element = driver.find_element_by_id("DdlstFinYear")
            year_drp = Select(year_element)
            year_drp.select_by_visible_text(i)

            slct_block()

            driver.find_element_by_id('TxtBox6').click()

            drp_ele = driver.find_element_by_id('DdlstTxtBox6')
            drp = Select(drp_ele)
            drp.select_by_visible_text(l)

            driver.find_element_by_id("viewDummy").click()
            time.sleep(2) 

            html=driver.page_source
            soup=BeautifulSoup(html,'html.parser')
            div=soup.select_one("div#content")
            table=pd.read_html(str(div))
            total_persons_demanded_work_bl.append(table)
            driver.find_element_by_id("BtnBack").click()

    #total_hh_worked
    total_hh_worked_bl=[]
    for i in year_slct:    
        for l in months:

            year_element = driver.find_element_by_id("DdlstFinYear")
            year_drp = Select(year_element)
            year_drp.select_by_visible_text(i)

            slct_block()

            driver.find_element_by_id('TxtBox7').click()

            drp_ele = driver.find_element_by_id('DdlstTxtBox7')
            drp = Select(drp_ele)
            drp.select_by_visible_text(l)

            driver.find_element_by_id("viewDummy").click()
            time.sleep(2) 

            html=driver.page_source
            soup=BeautifulSoup(html,'html.parser')
            div=soup.select_one("div#content")
            table=pd.read_html(str(div))
            total_hh_worked_bl.append(table)
            driver.find_element_by_id("BtnBack").click()

    #total_persons_worked
    total_persons_worked_bl=[]
    for i in year_slct:    
        for l in months:

            year_element = driver.find_element_by_id("DdlstFinYear")
            year_drp = Select(year_element)
            year_drp.select_by_visible_text(i)

            slct_block()

            driver.find_element_by_id('TxtBox8').click()

            drp_ele = driver.find_element_by_id('DdlstTxtBox8')
            drp = Select(drp_ele)
            drp.select_by_visible_text(l)

            driver.find_element_by_id("viewDummy").click()
            time.sleep(2) 

            html=driver.page_source
            soup=BeautifulSoup(html,'html.parser')
            div=soup.select_one("div#content")
            table=pd.read_html(str(div))
            total_persons_worked_bl.append(table)
            driver.find_element_by_id("BtnBack").click()

    driver.quit()

    '''df_total_hh_completed_person_days_bl = pd.DataFrame(total_hh_completed_person_days_bl)
    df_total_hh_completed_person_days_bl.to_csv('df_total_hh_completed_person_days_bl.csv') 

    df_total_workers_employed_bl = pd.DataFrame(total_workers_employed_bl)
    df_total_workers_employed_bl.to_csv('df_total_workers_employed_bl.csv')

    df_total_hh_demanded_work_bl = pd.DataFrame(total_hh_demanded_work_bl)
    df_total_hh_demanded_work_bl.to_csv('df_total_hh_demanded_work_bl.csv')

    df_total_persons_demanded_work_bl = pd.DataFrame(total_persons_demanded_work_bl)
    df_total_persons_demanded_work_bl.to_csv('df_total_persons_demanded_work_bl.csv')

    df_total_hh_worked_bl = pd.DataFrame(total_hh_worked_bl)
    df_total_hh_worked_bl.to_csv('df_total_hh_worked_bl.csv')

    df_total_persons_worked_bl = pd.DataFrame(total_persons_worked_bl)
    df_total_persons_worked_bl.to_csv('df_total_persons_worked_bl.csv')'''

    return df_state
    pkl_df_state = "df_state.pkl"
    path = os.path.join(download_dir, pkl_df_state)
    df_state.to_pickle(path)

    return df_district
    pkl_df_district = "df_district.pkl"
    path = os.path.join(download_dir, pkl_df_district)
    df_district.to_pickle(path)

    return df_block
    pkl_df_block = "df_block.pkl"
    path = os.path.join(download_dir, pkl_df_block)
    df_block.to_pickle(path)

    #state-level for remaining indicators
    return df_total_hh_completed_person_days_st
    pkl_df_total_hh_completed_person_days_st = "df_total_hh_completed_person_days_st.pkl"
    path = os.path.join(download_dir, pkl_df_total_hh_completed_person_days_st)
    df_total_hh_completed_person_days_st.to_pickle(path)

    return df_total_workers_employed_st
    pkl_df_total_workers_employed_st = "df_total_workers_employed_st.pkl"
    path = os.path.join(download_dir, pkl_df_total_workers_employed_st)
    df_total_workers_employed_st.to_pickle(path)

    return df_total_hh_demanded_work_st
    pkl_df_total_hh_demanded_work_st = "df_total_hh_demanded_work_st.pkl"
    path = os.path.join(download_dir, pkl_df_total_hh_demanded_work_st)
    df_total_hh_demanded_work_st.to_pickle(path)

    return df_total_persons_demanded_work_st
    pkl_df_total_persons_demanded_work_st = "df_total_persons_demanded_work_st.pkl"
    path = os.path.join(download_dir, pkl_df_total_persons_demanded_work_st)
    df_total_persons_demanded_work_st.to_pickle(path)

    return df_total_hh_worked_st
    pkl_df_total_hh_worked_st = "df_total_hh_worked_st.pkl"
    path = os.path.join(download_dir, pkl_df_total_hh_worked_st)
    df_total_hh_worked_st.to_pickle(path)

    return df_total_persons_worked_st
    pkl_df_total_persons_worked_st = "df_total_persons_worked_st.pkl"
    path = os.path.join(download_dir, pkl_df_total_persons_worked_st)
    df_total_persons_worked_st.to_pickle(path)

    #district-level for remaining indicators
    return df_total_hh_completed_person_days_dt
    pkl_df_total_hh_completed_person_days_dt = "df_total_hh_completed_person_days_dt.pkl"
    path = os.path.join(download_dir, pkl_df_total_hh_completed_person_days_dt)
    df_total_hh_completed_person_days_dt.to_pickle(path)

    return df_total_workers_employed_dt
    pkl_df_total_workers_employed_dt = "df_total_workers_employed_dt.pkl"
    path = os.path.join(download_dir, pkl_df_total_workers_employed_dt)
    df_total_workers_employed_dt.to_pickle(path)

    return df_total_hh_demanded_work_dt
    pkl_df_total_hh_demanded_work_dt = "df_total_hh_demanded_work_dt.pkl"
    path = os.path.join(download_dir, pkl_df_total_hh_demanded_work_dt)
    df_total_hh_demanded_work_dt.to_pickle(path)

    return df_total_persons_demanded_work_dt
    pkl_df_total_persons_demanded_work_dt = "df_total_persons_demanded_work_dt.pkl"
    path = os.path.join(download_dir, pkl_df_total_persons_demanded_work_dt)
    df_total_persons_demanded_work_dt.to_pickle(path)

    return df_total_hh_worked_dt
    pkl_df_total_hh_worked_dt = "df_total_hh_worked_dt.pkl"
    path = os.path.join(download_dir, pkl_df_total_hh_worked_dt)
    df_total_hh_worked_dt.to_pickle(path)

    return df_total_persons_worked_dt
    pkl_df_total_persons_worked_dt = "df_total_persons_worked_dt.pkl"
    path = os.path.join(download_dir, pkl_df_total_persons_worked_dt)
    df_total_persons_worked_dt.to_pickle(path)

    #block-level for remaining indicators
    return df_total_hh_completed_person_days_bl
    pkl_df_total_hh_completed_person_days_bl = "df_total_hh_completed_person_days_bl.pkl"
    path = os.path.join(download_dir, pkl_df_total_hh_completed_person_days_bl)
    df_total_hh_completed_person_days_bl.to_pickle(path)

    return df_total_workers_employed_bl
    pkl_df_total_workers_employed_bl = "df_total_workers_employed_bl.pkl"
    path = os.path.join(download_dir, pkl_df_total_workers_employed_bl)
    df_total_workers_employed_bl.to_pickle(path)

    return df_total_hh_demanded_work_bl
    pkl_df_total_hh_demanded_work_bl = "df_total_hh_demanded_work_bl.pkl"
    path = os.path.join(download_dir, pkl_df_total_hh_demanded_work_bl)
    df_total_hh_demanded_work_bl.to_pickle(path)

    return df_total_persons_demanded_work_bl
    pkl_df_total_persons_demanded_work_bl = "df_total_persons_demanded_work_bl.pkl"
    path = os.path.join(download_dir,pkl_df_total_persons_demanded_work_bl )
    df_total_persons_demanded_work_bl.to_pickle(path)

    return df_total_hh_worked_bl
    pkl_df_total_hh_worked_bl = "df_total_hh_worked_bl.pkl"
    path = os.path.join(download_dir,pkl_df_total_hh_worked_bl )
    df_total_hh_worked_bl.to_pickle(path)

    return df_total_persons_worked_bl
    pkl_df_total_persons_worked_bl = "df_total_persons_worked_bl.pkl"
    path = os.path.join(download_dir,pkl_df_total_persons_worked_bl )
    df_total_persons_worked_bl.to_pickle(path)

ValueError: No tables found

KeyboardInterrupt: 

In [24]:

df[0][1]

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,NaN,NaN,MGNREGA Public Data Portal For FY: 2012-2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,Sno.State NameTotal households applied for job...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Sno.,State Name,Total households applied for job card,Total job cards issued,Total job cards (SC),Total job cards (ST),Total job cards (non-SC/ST),Total households demanded work,Total persons demanded work,Total households allotted work,...,Amount sanctioned (Rs. In Lakhs),Total works,Total bank accounts,Total individual bank accounts,Total joint bank accounts,Amount disbursed to bank accounts,Total post office accounts,Total individual post office accounts,Total joint post office accounts,Amount disbursed to post office accounts
5,1,ANDAMAN AND NICOBAR,48489,48485,4,3976,44505,18212,21840,18238,...,107424461,1402,11218,10459,759,14232851,622,619,3,258939
6,2,DADRA & NAGAR HAVELI,8176,3554,1,3499,54,4,4,3,...,22,11,2181,2172,9,1099,0,0,0,0
7,3,DAMAN & DIU,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,4,LAKSHADWEEP,8493,8424,8,8321,95,1963,2076,1961,...,1006,191,1253,1249,4,1801552,6470,6445,25,7765221
9,5,NAGALAND,398035,397829,3545,372154,22130,387007,411424,387006,...,245722258,15588,71,68,3,0,1,1,0,0


In [ ]:
slct_district()
district_table
district_table.to_csv('district_manrega.csv') 

In [ ]:
slct_block()
block_table
block_table.to_csv('block_manrega.csv') 

In [23]:
state_table[0][1].head()

,Sno.,State Name,Total households applied for job card,Total job cards issued,Total job cards (SC),Total job cards (ST),Total job cards (non-SC/ST),Total households demanded work,Total persons demanded work,Total households allotted work,...,Amount sanctioned (Rs. In Lakhs),Total works,Total bank accounts,Total individual bank accounts,Total joint bank accounts,Amount disbursed to bank accounts,Total post office accounts,Total individual post office accounts,Total joint post office accounts,Amount disbursed to post office accounts
0,1.0,ANDAMAN AND NICOBAR,45396,45396,4,2555,42837,19912,23586,19905,...,44677804,1240,11532,10801,731,25418828,942,934,8,883274
1,2.0,DADRA & NAGAR HAVELI,7845,1726,1,1703,22,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,3.0,DAMAN & DIU,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4.0,LAKSHADWEEP,8254,8162,8,8062,92,3891,4290,3889,...,934,135,159,159,0,723681,5982,5956,26,18003310
4,5.0,NAGALAND,384478,384478,3530,353590,27358,370590,402015,370521,...,93813130,27687,96550,63136,33414,0,1,1,0,0


In [115]:
state_1 = state_table[1][1]

In [117]:
select_indicators()

NameError: name 'select_indicators' is not defined

In [149]:
wait.until(EC.element_to_be_clickable((By.ID, check_list))).click()

TimeoutException: Message: 


In [ ]:
check_list = ['ChkLstFieldsWorkerA_0', 'ChkLstFieldsWorkerA_1','ChkLstFieldsWorkerA_2','ChkLstFieldsWorkerA_3','ChkLstFieldsWorkerA_4',
             'ChkLstFieldsWorkerC_0', 'ChkLstFieldsWorkerC_1', 'ChkLstFieldsWorkerD_0', 'ChkLstFieldsWorkerD_1',
              'ChkLstFieldsWorkerE_0','ChkLstFieldsWorkerE_1','ChkLstFieldsWorkerE_2','ChkLstFieldsWorkerE_3',
              'ChkLstFieldsWorkerE_4','ChkLstFieldsWorkerE_5','ChkLstFieldsWorkerE_6', 
             'ChkLstFieldsWorkerE_7','ChkLstFieldsWorkerE_8','ChkLstFieldsWorkerE_9','ChkLstFieldsWorkerE_10',
              'ChkLstFieldsWorkerE_11','ChkLstFieldsWorkerE_12','ChkLstFieldsWorkerE_13',
              'CTxtBox1', 'CTxtBox2', 
             'ChkLstFieldsWork_0', 'ChkLstFieldsWork_1','ChkLstFieldsWork_2','ChkLstFieldsWork_3',
             'ChkLstFieldsWork_W_0','ChkLstFieldsWork_W_1',
             'ChkBox1','ChkBox2','ChkBox4',
             'CTxtBox3','CTxtBox4','CTxtBox5','CTxtBox6','CTxtBox7','CTxtBox8','CTxtBox9','CTxtBox10']
year_slct = ['2011-2012', '2012-2013','2013-2014','2014-2015','2015-2016','2016-2017','2017-2018','2018-2019','2019-2020','2020-2021']
lod_slct = ['State', 'District', 'Block']

def slct_indicators():
    for i in check_list:
        checkboxes = driver.find_elements_by_id(i)
        for checkbox in checkboxes:
            while not checkbox.is_selected():
                checkbox.click()
                
def slct_state():
    lod_element = driver.find_element_by_id('regionselect')
    lod_drp = Select(lod_element)
    lod_drp.select_by_visible_text('State')
    driver.find_element_by_xpath("//*[@id='checklist']/ul/li/label/input").click()
    
def slct_district():
    lod_element = driver.find_element_by_id('regionselect')
    lod_drp = Select(lod_element)
    lod_drp.select_by_visible_text('District')
    driver.find_element_by_xpath("//*[@id='checklist']/ul/li/label/input").click()
    
def slct_block():
    lod_element = driver.find_element_by_id('regionselect')
    lod_drp = Select(lod_element)
    lod_drp.select_by_visible_text('Block')
    driver.find_element_by_xpath("//*[@id='checklist']/ul/li/label/input").click()

year_element = driver.find_element_by_id("DdlstFinYear")
year_drp = Select(year_element)
year_drp.select_by_visible_text('Select')
time.sleep(5)


def state_table():
    lst_state = []
    for i in year_slct:
        slct_indicators()
        slct_state()

        year_element = driver.find_element_by_id("DdlstFinYear")
        year_drp = Select(year_element)
        year_drp.select_by_visible_text(i)

        driver.find_element_by_id("viewDummy").click()
        time.sleep(5) 
        tbl = driver.find_element_by_xpath("//*[@id='content']/table/tbody").get_attribute('outerHTML')
        df = pd.read_html(tbl)
        lst_state.append(df)
        
        driver.find_element_by_id("BtnBack").click()
